In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("./IMDB Dataset.csv")
data.head()

In [ ]:
data['sentiment'] = np.where(data['sentiment'] ==  'positive', 1, 0)
data.head()

In [ ]:
reviews = data['review'].to_numpy()
labels = data['sentiment'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.2, random_state=0)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)

train_sequence = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(train_sequence, maxlen=200, padding="post", truncating="post")

test_sequence = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequence, maxlen=200, padding="post", truncating="post")

In [ ]:
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

bi_lstm_model = tf.keras.models.Sequential([
    # 10000: vocablulary size, 128: embedding dimension, 200: sequence length, input length
    Embedding(10000, 128, input_length=200),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.05),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.05),
    Bidirectional(LSTM(128)),
    Dense(1, activation="sigmoid")
])
bi_lstm_model.compile(optimizer="adam", loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
bi_lstm_model.fit(train_padded, y_train, epochs=10)

In [ ]:
bi_lstm_model.evaluate(test_padded, y_test, verbose=2)